# IFC Rasterization for Efficient Spatial Analysis

## Intro

### Import libraries and the IFC model:

## Functions

In [3]:
import pyvista as pv
import numpy as np
from typing import Any, Tuple
import ifcopenshell
from importlib import reload 
import multiprocessing
import ifcopenshell
import ifcopenshell.geom
import time
from functools import reduce
import vtk
ifc_file = ifcopenshell.open(r"IFC Files\Project1.ifc")
#ifc_file = ifcopenshell.open(r"IFC Files\Duplex.ifc")
#ifc_file = ifcopenshell.open(r"IFC Files\Clinic.ifc")

def to_vtk_faces(faces : Tuple[tuple]) -> np.ndarray:

    faces=np.array(faces, dtype=np.int16)
    num_insertions = (len(faces) - 1) // 3

    # Generate an array of indices for insertions
    indices = np.arange(3, 3 * (num_insertions + 1), 3)
    indices = np.insert(indices, 0, 0)
    faces = np.insert(faces, indices, 3) 
    return faces

def vtk_block_by_building_element(ifc_file):
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.APPLY_DEFAULT_MATERIALS, True)
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    exclude_list = ["IfcSpace", "IfcOpeningElement"]
    multiblock = pv.MultiBlock()

    if iterator.initialize():
        polydata_list = []
        while True:
            shape = iterator.get()
            if shape.type not in exclude_list:
                faces = shape.geometry.faces
                verts = shape.geometry.verts
                poly_data = pv.PolyData(list(verts), to_vtk_faces(faces))
                multiblock.append(poly_data)
                #polydata_list.append(poly_data)
                
            if not iterator.next():
                break

    #merged = reduce(lambda acc, x: acc.merge(x), polydata_list)
               
    return multiblock

def create_uniform_grid(bounds, voxel_size):
    """Create a uniform grid within the given bounds."""
    x = np.arange(bounds[0], bounds[1] + voxel_size, voxel_size)
    y = np.arange(bounds[2], bounds[3] + voxel_size, voxel_size)
    z = np.arange(bounds[4], bounds[5] + voxel_size, voxel_size)
    return pv.StructuredGrid(*np.meshgrid(x, y, z))

def voxelize_space(meshes, voxel_size):
    
    """Voxelize space and check intersections with given mesh."""

    grid = create_uniform_grid(meshes.bounds, voxel_size)

    num_points = grid.cell_centers().n_points

    mask = np.zeros(num_points, dtype=bool)

    print(f'Total number of voxels: {num_points}')

    select_enclosed = vtk.vtkSelectEnclosedPoints()
    select_enclosed.SetInputData(grid.cell_centers())

    #lambda function to check each mesh in the multiblock against the grid
    for mesh in meshes:
        select_enclosed.SetSurfaceData(mesh)
        select_enclosed.Update()
        # write a lambda function to check if a point is inside the mesh
        # and use it to set the mask
        mask = np.logical_or(mask, np.array([select_enclosed.IsInside(i) for i in range(num_points)]))
    
    return grid, mask


## Voxelization

In [4]:
# convert IFC objects into vtk meshes

start_time = time.time()

all_meshes = vtk_block_by_building_element(ifc_file)

end_time = time.time()
print(f"Time taken to convert objects: {end_time - start_time:.4f} seconds")

Time taken to convert objects: 0.2380 seconds


In [5]:
# Voxelize the entire space and check for intersections with the meshes
voxel_size = 0.2

start_time = time.time()

grid, mask = voxelize_space(all_meshes, voxel_size)

end_time = time.time()
print(f"Time taken to voxelize: {end_time - start_time:.4f} seconds")

# Visualization
p = pv.Plotter()
p.add_mesh(grid, opacity=0.3, show_edges=True) 
p.add_mesh(grid.extract_cells(np.where(mask)[0]), color="blue", opacity=0.5)
p.show()

Total number of voxels: 10944
Time taken to voxelize: 0.0656 seconds


Widget(value="<iframe src='http://localhost:54680/index.html?ui=P_0x2749863bd60_0&reconnect=auto' style='width…